In [9]:
pip install -Uqq rdflib

In [3]:
pip install -Uqq ipykernel

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -Uqq rdflib

Note: you may need to restart the kernel to use updated packages.


In [202]:
import pandas as pd
import pickle
from rdflib import Graph, Literal, RDF, URIRef
from rdflib import Namespace
import uuid

In [203]:
# tmp = pickle.load(open('data/irs-filings/processed_lists.pickle','rb'))
# tmp.keys()
# df8871 = pd.DataFrame(tmp['1']).sample(500).copy()
# directorsdf = pd.DataFrame(tmp['D']).copy()
# directorsdf = directorsdf[directorsdf.ein.isin(df8871.ein)]
# df8871.to_pickle('8871sample.pickle')
# directorsdf.to_pickle('directorssample.pickle')

In [204]:
df8871 = pd.read_pickle('8871sample.pickle')
directorsdf = pd.read_pickle('directorssample.pickle')

In [205]:
# display(df8871.head(1))
directorsdf = directorsdf[['form_id_number', 'director_id', 'ein',
                           'entity_name', 'entity_title', 'entity_address_1', 'entity_address_2',
                           'entity_address_city', 'entity_address_st', 'entity_address_zip_code',
                           'entity_address_zip_code_ext']]
display(directorsdf.head(1))

,form_id_number,director_id,ein,entity_name,entity_title,entity_address_1,entity_address_2,entity_address_city,entity_address_st,entity_address_zip_code,entity_address_zip_code_ext
787,305.0,42980.0,011111111,RANDY POMBO,TRUSTEE,28375 S. CHRISMAN RD.,None,TRACY,CA,95304,None


In [206]:
parent_things_config = {'form_id_number':'document:form8871','ein':'organization:527'}
main_thing_config = {'director_id':'person'}

In [207]:
def get_uid(): return uuid.uuid1().hex

def get_uids(df,column,prefix):
    unique_values = df[column].unique()
    uids = {o:URIRef(f"{prefix}:{get_uid()}") for o in unique_values}
    return uids

In [208]:
parent_uids,main_uids = {},{}

for key in parent_things_config:
    parent_uids[key] = get_uids(directorsdf,key,parent_things[key])
    
key = [o for o in main_thing_config.keys()][0]
main_uids[key] = get_uids(directorsdf,key,main_thing_config[key])

In [209]:
g = Graph()

main_thing = 'director_id'
value = Namespace('value')
all_things = [main_thing] + [o[0] for o in parent_things]
for index,row in enumerate(directorsdf.iterrows()):
    center_node_ref = f"{'director_id'}:{main_uids['director_id'][row[1]['director_id']]}"
    center_node = URIRef(center_node_ref)
    
    for field in row[1].iteritems():
        if field[0] in parent_things.keys():
            parent_node = f"{parent_uids[field[0]][field[1]]}"
            tmp_thing = URIRef(parent_node)
            g.add((center_node,URIRef(f"value_{field[0]}"),tmp_thing))
        else:
            g.add((center_node,URIRef(f"value_{field[0]}"),Literal(field[1])))

g.bind("val", value)

# print(g.serialize(format='n3'))

In [210]:
g.serialize(destination='director.txt', format='turtle')

<Graph identifier=N0490056607674db0bda7750f9d4d7b9d (<class 'rdflib.graph.Graph'>)>